In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv('data/Mental Illness Survey 1.csv')
data.head()

,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,I identify as having a mental illness,Education,I have my own computer separate from a smart phone,I have been hospitalized before for my mental illness,How many days were you hospitalized for your mental illness,I am currently employed at least part-time,I am legally disabled,I have my regular access to the internet,I live with my parents,I have a gap in my resume,Total length of any gaps in my resume in months.,Annual income (including any social welfare programs) in USD,I am unemployed,I read outside of work and school,Annual income from social welfare programs,I receive food stamps,I am on section 8 housing,How many times were you hospitalized for your mental illness,I have one of the following issues in addition to my illness,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Age,Gender,Household Income,Region,Device Type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response,Response,Response,Response,Open-Ended Response,Response,Response,Response,Response,Response,Open-Ended Response,Open-Ended Response,Response,Response,Open-Ended Response,Response,Response,Open-Ended Response,Lack of concentration,Anxiety,Depression,Obsessive thinking,Mood swings,Panic attacks,Compulsive behavior,Tiredness,Response,Response,Response,Response,Response
1,6.630447e+09,168522804.0,01/15/2018 03:45:16 AM,01/15/2018 03:48:24 AM,NaN,NaN,NaN,NaN,06f645d7ea5af372d50a62bd17,No,High School or GED,No,No,0,No,No,Yes,No,Yes,24,35,Yes,Yes,0,No,No,0,Lack of concentration,Anxiety,Depression,Obsessive thinking,NaN,Panic attacks,NaN,NaN,30-44,Male,"$25,000-$49,999",Mountain,Android Phone / Tablet
2,6.630410e+09,168522804.0,01/15/2018 03:17:52 AM,01/15/2018 03:18:57 AM,NaN,NaN,NaN,NaN,abca2776418ff1fe24bb85e21f,Yes,Some Phd,Yes,No,0,Yes,No,Yes,No,No,1,22,No,Yes,0,No,No,0,Lack of concentration,Anxiety,Depression,NaN,NaN,Panic attacks,NaN,Tiredness,18-29,Male,"$50,000-$74,999",East South Central,MacOS Desktop / Laptop
3,6.630402e+09,168522804.0,01/15/2018 03:10:28 AM,01/15/2018 03:12:49 AM,NaN,NaN,NaN,NaN,3800088cf4e55278b38bbe67f3,No,Completed Undergraduate,Yes,No,0,Yes,No,Yes,No,No,0,100,No,Yes,0,No,No,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30-44,Male,"$150,000-$174,999",Pacific,MacOS Desktop / Laptop
4,6.630335e+09,168522804.0,01/15/2018 02:11:16 AM,01/15/2018 02:12:33 AM,NaN,NaN,NaN,NaN,84585803a3cec189f89fe43d44,No,Some Undergraduate,Yes,No,NaN,No,No,Yes,Yes,Yes,11,0,Yes,Yes,0,No,No,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30-44,Male,"$25,000-$49,999",New England,Windows Desktop / Laptop


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 40 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Respondent ID                                                 334 non-null    float64
 1   Collector ID                                                  334 non-null    float64
 2   Start Date                                                    334 non-null    object 
 3   End Date                                                      334 non-null    object 
 4   IP Address                                                    0 non-null      float64
 5   Email Address                                                 0 non-null      float64
 6   First Name                                                    0 non-null      float64
 7   Last Name                                                     0 non-nul

# Cleaning & Preprocessing
So we see that this data needs quite a bit of cleaning.
- deal with secondary header
- unnamed cols
- cols to drop
- Handle nulls
- convert cols to appropriate dtype
- a lot of categorical data
    - label encoding
    - one hot encoding

## Rename & Drop cols
- The collector id is the same for all records, so drop. 
- I don't need the respondent id since they are all unique and we can use index as the id
- IP Address, Email Address, First Name, and Last Name are all full of nulls
- I have no definition for 'Custom Data 1' and it is not apparent what it represents
- Some features seem to be asking very similar things and are likely colinear
    - 'I have been hospitalized before for my mental illness' and 'How many days were you hospitalized for your mental illness"
    - 'I have a gap in my resume' and 'Total length of any gaps in my resume in months.'
    - 'I am currently employed at least part-time' and 'I am unemployed'
- consider dropping 'Annual income (including any social welfare programs) in USD' and 'Annual income from social welfare programs' if target is about unemployment since these cols may lead to an overfit model
    


In [4]:
# Name Unnamed cols
data.rename(columns={'Unnamed: 28':'Anxiety',
                     'Unnamed: 29':'Depression',
                     'Unnamed: 30':'Obsessive_thinking',
                     'Unnamed: 31':'Mood_swings',
                     'Unnamed: 32':'Panic_attacks',
                     'Unnamed: 33':'Compulsive_behavior',
                     'Unnamed: 34':'Tiredness',
                     data.columns[27] : 'Lack of concentration',
                     data.columns[19] : 'Total gaps in resume in months', # this col name was causing problems so need to rename
                    }, inplace=True)

In [29]:
data.drop(['Respondent ID', 'Collector ID', 'IP Address', 'Email Address', 'First Name', 'Last Name', 'Custom Data 1'], 
          axis=1, inplace=True)

In [30]:
# drop row 0 since it is a secondary header we don't need
data.drop(index=0, axis=0, inplace=True)

In [31]:
# Handle nulls
for col in data.columns:
    if data[col].isnull().sum() != 0:
        print(f'{col} : {data[col].isnull().sum()}')

How many days were you hospitalized for your mental illness : 37
Lack of concentration : 283
Anxiety : 234
Depression : 248
Obsessive_thinking : 292
Mood_swings : 296
Panic_attacks : 285
Compulsive_behavior : 305
Tiredness : 234
Region : 2


## In order to do analysis of the nulls, I need to convert the cols to the appropriate dtypes

I am most interested in 'How many days hospitalized...' and 'Region' since the other cols null mean the user didn't endorse that symptom. These will later be converted to 0s to reflect this.

So do I impute '..days hospitalized..' or drop? How about comparing it to 'I have been hospitalized before for my mental illness' and if they say they say no to this, replace null with 0. 

In [5]:
data['I have been hospitalized before for my mental illness'][data['How many days were you hospitalized for your mental illness'].isna()]

4       No
11      No
16      No
20      No
28      No
30      No
32      No
36      No
46      No
64      No
91      No
98      No
110     No
112     No
118     No
127     No
163     No
176     No
183     No
186     No
199     No
203     No
206     No
208     No
226     No
232     No
234     No
250     No
264     No
266    Yes
271     No
282     No
283     No
293     No
298     No
315     No
330     No
Name: I have been hospitalized before for my mental illness, dtype: object

In [28]:
data[(data['I have been hospitalized before for my mental illness']=='No') & (data['How many days were you hospitalized for your mental illness']>0)]

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
mask1 = data['I have been hospitalized before for my mental illness']=='Yes'
mask2 = data['How many days were you hospitalized for your mental illness'].isna()
mask2

In [ ]:
data.head()